In [1]:
import tifffile
import numpy as np
import skimage
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pumapy as puma
import pyvista as pv
import os
import pandas as pd
import math

'import TexGen.Core' failed (TexGen is only made available when installing puma with conda on UNIX).


In [2]:
def puma_permeability (binary):
    ws = puma.Workspace.from_array(binary.copy())
    #print(f"Shape of workspace: {ws.matrix.shape}")
    keff = puma.compute_permeability(ws, solid_cutoff=(1, 1),
                                                      solver_type='minres',
                                                      direction='xyz', tol=1e-04,
                                                      maxiter=10000,display_iter=True,
                                                      matrix_free=True, precondition=False,
                                                      output_fields=False)
    return keff #Effective permeability tensor

In [3]:
def puma_specific_area (binary):
    ws = puma.Workspace.from_array(binary.copy())
    #print(f"Shape of workspace: {ws.matrix.shape}")
    area_us, specific_area_us = puma.compute_surface_area(ws, cutoff=(1,1))
    
    return  specific_area_us


In [4]:
def puma_mean_intercept_length(binary):
    ws = puma.Workspace.from_array(binary.copy())
    mil = puma.compute_mean_intercept_length(ws, void_cutoff=(1, 1))
    return mil

In [5]:
def puma_tortuosity (binary):
    ws = puma.Workspace.from_array(binary.copy())
    #print(f"Shape of workspace: {ws.matrix.shape}")    
    n_eff_x, Deff_x, poro, C_x = puma.compute_continuum_tortuosity(ws, (0,0), 'x', side_bc='s', tolerance=1e-4, solver_type='cg')
    n_eff_y, Deff_y, poro, C_y = puma.compute_continuum_tortuosity(ws, (0,0), 'y', side_bc='s', tolerance=1e-4, solver_type='cg')
    n_eff_z, Deff_z, poro, C_z = puma.compute_continuum_tortuosity(ws, (0,0), 'z', side_bc='s', tolerance=1e-4, solver_type='cg')

   # poro is the porosity of the material
   # n_eff is the effective tortuosity factor
   # C_x is the computed field vector 
    return n_eff_x, n_eff_y , n_eff_z


In [6]:
def puma_orientations (binary):
    ors = puma.Workspace.from_array(binary.copy())
    print(f"Shape of workspace: {ors.matrix.shape}")
    puma.compute_orientation_st(ors, cutoff=(1,1), sigma=1.4, rho=0.7, edt=True)
    ors_z = ors.orientation[:,:,:,0].mean()
    ors_y = ors.orientation[:,:,:,1].mean()
    ors_x = ors.orientation[:,:,:,2].mean()
    
    return round(ors_x,2), round(ors_y,2), round(ors_z,2)

In [7]:
def binary_seg_kMeans(img):
    binary = np.zeros_like(img)
    pixels = img.reshape(-1, 1)
    kmeans = KMeans(n_clusters=2)
    kmeans.fit(pixels)
    centers = kmeans.cluster_centers_
    thresh = (centers[0] + centers[1])/2
    binary[img > thresh] = 1
    return binary

In [8]:
def contrast_stretching(input_image):
    #Contrast stretching
    #Dropping extreems (artifacts)
    p2, p98 = np.percentile(input_image, (2, 98))
    stretched_image = skimage.exposure.rescale_intensity(input_image, in_range=(p2, p98))
    return stretched_image.astype('uint8')

In [9]:
def read_and_preprocessing (path, s):
    
    img = tifffile.imread(path)
    img = img[s[0]:s[1],s[2]:s[3],s[4]:s[5]]
    img = contrast_stretching(img).astype('uint8')
    
    return img
    

In [10]:
def pixel_metrics(pred, target):
    
    mse = skimage.metrics.mean_squared_error(target, pred)
    rmse = math.sqrt(mse)
    #mi = skimage.metrics.normalized_mutual_information(target, pred, bins=100)
    psnr = skimage.metrics.peak_signal_noise_ratio(target, pred, data_range=None)
    ssi = skimage.metrics.structural_similarity(target, pred, win_size=None, gradient=False,
                                                data_range=None, channel_axis=None,
                                                gaussian_weights=False, full=False,)
    
    return {'rmse':rmse, 'psnr':psnr, 'ssi':ssi}

In [11]:
def microstructure_metric(img):
    ice_density = 0.92 #(g/cm³)
    pixel_volume = (0.006)**3 # cm³
    voxel = 0.006 #cm
    
    relative_density = round(len(img[img>0])/img.size,2)
    density = relative_density * ice_density
    
    porosity = round (len (img[img==0])*100/img.size,2)
    
    verts, faces, _, _ = skimage.measure.marching_cubes(img, level=0)
    surface_area = skimage.measure.mesh_surface_area(verts, faces)
    SSA =round( (surface_area * voxel * voxel),2) # mm^2
    
    #For 3D objects, the Euler number is obtained as the number of objects
    #plus the number of holes, minus the number of tunnels, or loops.
    euler = skimage.measure.euler_number(img, connectivity=1)
    
    return {'density':density, 'porosity':porosity, 'SSA':SSA, 'euler':euler}

In [12]:
snow_slice = [10,210,800,1000,200,400]
firn_slice = [10,210,600,800,1200,1400]
ice_slice= [10,210,200,400,800,1000]

In [13]:
snow_files = {'High-res': 'data/registered_image02_3dCT_B40_bag12_13_300mm.tif',
              'Bicubic' : 'data/02_Substack (6267-6662)_B40_bag12_13.tif',
              'SRCNN'   : 'model_output/02_Substack_predictions_SRCNN_.tif',
              'DCSRN'   : 'model_output/02_Substack_predictions_DCSRN_.tif',
              'SRUnet'  : 'model_output/02_Substack_predictions_SRUnet_.tif',
              'SRResnet': 'model_output/02_Substack_predictions_SRResnet_.tif'}

In [14]:
firn_files = {'High-res': 'data/registered_image06_3dCT_B40_bag56_57_100mm.tif',
              'Bicubic' : 'data/06_Substack (8055-8449)_B40_bag56_57.tif',
              'SRCNN'   : 'model_output/06_Substack_predictions_SRCNN_.tif',
              'DCSRN'   : 'model_output/06_Substack_predictions_DCSRN_.tif',
              'SRUnet'  : 'model_output/06_Substack_predictions_SRUnet_.tif',
              'SRResnet': 'model_output/06_Substack_predictions_SRResnet_.tif'}


In [15]:
ice_files = {'High-res': 'data/registered_image10_3dCT_B40_bag108_109_538mm.tif',
              'Bicubic' : 'data/10_Substack (4268-4663)_B40_bag108_109.tif',
              'SRCNN'   : 'model_output/10_Substack_predictions_SRCNN_.tif',
              'DCSRN'   : 'model_output/10_Substack_predictions_DCSRN_.tif',
              'SRUnet'  : 'model_output/10_Substack_predictions_SRUnet_.tif',
              'SRResnet': 'model_output/10_Substack_predictions_SRResnet_.tif'}

In [16]:
pixel_result = pd.DataFrame(columns=['type','name','rmse','psnr','ssi'])



In [17]:
pixel_result.head()

,type,name,rmse,psnr,ssi


In [18]:
for num , f in enumerate([snow_files,firn_files,ice_files]):
    type_ = ['snow','firn','ice'][num]
    print (type_)
    slice_ = [snow_slice,firn_slice,ice_slice][num]
    target = read_and_preprocessing (f['High-res'], slice_)
    for row_number , name in enumerate(f.keys()):
        row_number = num*6 + row_number
        #print (name)
        #print (row_number)
        data = read_and_preprocessing (f[name], slice_)
        pixel_result.at[row_number, 'type'] = type_
        pixel_result.at[row_number, 'name'] = name
        pm = pixel_metrics(data,target)
        pixel_result.at[row_number, 'rmse'] =round( pm['rmse'],2)        
        pixel_result.at[row_number, 'psnr'] =  round(pm['psnr'],2)
        pixel_result.at[row_number, 'ssi'] =  round(pm['ssi'],2)
    
    



snow


C:\Users\Fabagh001\Anaconda3\envs\puma_env\lib\site-packages\skimage\metrics\simple_metrics.py:160: RuntimeWarning: divide by zero encountered in scalar divide
  return 10 * np.log10((data_range ** 2) / err)


firn
ice


In [21]:
pixel_result.head(2)

,type,name,rmse,psnr,ssi
0,snow,High-res,0.0,inf,1.0
1,snow,Bicubic,44.57,15.15,0.68


In [30]:
microstructur_result = pd.DataFrame(columns=['type','name','density','porosity','SSA','euler','mil',
                                              'orientation','tortuosity','permeability'])

In [31]:
for num , f in enumerate([snow_files,firn_files,ice_files]):
    type_ = ['snow','firn','ice'][num]
    print (type_)
    slice_ = [snow_slice,firn_slice,ice_slice][num]
    for row_number , name in enumerate(f.keys()):
        row_number = num*6 + row_number
        print (name)
        print (row_number)
        data = read_and_preprocessing (f[name], slice_)
        data = binary_seg_kMeans (data)
        microstructur_result.at[row_number, 'type'] = type_
        microstructur_result.at[row_number, 'name'] = name
        mm = microstructure_metric(data)
        microstructur_result.at[row_number, 'density'] =round( mm['density'],2)        
        microstructur_result.at[row_number, 'porosity'] =  round(mm['porosity'],2)
        microstructur_result.at[row_number, 'SSA'] =  round(mm['SSA'],0)
        microstructur_result.at[row_number, 'euler'] =  round(mm['euler'],0)
        microstructur_result.at[row_number, 'mil'] =  str(puma_mean_intercept_length(data))
        microstructur_result.at[row_number, 'permeability'] =  str(puma_permeability(data))
        #microstructur_result.at[row_number, 'orientation'] =  str(puma_orientations (data))
        
        break
    break
        #microstructur_result.at[row_number, 'tortuosity'] =  str(puma_tortuosity(data))
        
        

snow
High-res
0
Approximate memory requirement for simulation: 655.15 MB
Initializing indexing matrices ... Done
Creating A matrix ... Done
Time to setup system: 86.27142869999989
Running x direction
Creating b vector ... Done
Solving Ax=b using minres solver
Iteration: 13, driving either residual (0.0000978338, 0.0006248611) --> target = 0.0001000000 ... Done
Running y direction
Creating b vector ... Done
Solving Ax=b using minres solver
Iteration: 14, driving either residual (0.0000884047, 0.0006230817) --> target = 0.0001000000 ... Done
Running z direction
Creating b vector ... Done
Solving Ax=b using minres solver
Iteration: 14, driving either residual (0.0000892063, 0.0006223485) --> target = 0.0001000000 ... Done

Effective permeability tensor: 
[[ 9.04822584e-13 -2.00813652e-15  2.79109805e-15]
 [-1.56830165e-15  9.02598804e-13 -1.23754860e-15]
 [ 2.45906405e-15 -1.24245279e-15  8.80446630e-13]]
Time to solve: 696.5947157



In [32]:
microstructur_result.head()

,type,name,density,porosity,SSA,euler,mil,orientation,tortuosity,permeability
0,snow,High-res,0.49,47.34,38.0,-1380,"(1.7589529989477374e-05, 1.7085700031637095e-0...",NaN,NaN,"(array([[ 9.04822584e-13, -2.00813652e-15, 2...."


In [33]:
sss = puma_mean_intercept_length(data)

In [34]:
sss

(1.7589529989477374e-05, 1.7085700031637095e-05, 1.6783383203114102e-05)